In [1]:
import pandas as pd
import numpy as np

import random
import os
import sys
import psutil

import matplotlib
import matplotlib.pyplot as plt
import math
from multiprocessing import cpu_count,Pool 
import multiprocessing

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Parallel Functions

In [2]:
class WithExtraArgs(object):
    def __init__(self, func, **args):
        self.func = func
        self.args = args
    def __call__(self, df):
        return self.func(df, **self.args)

def applyParallel(data, func,pool,partition, kwargs):
    data_split = [data[i:i + partition] for i in xrange(0, len(data), partition)]
    #data_split = np.array_split(data, min(partitions,data.shape[0]))
    data =pool.map(WithExtraArgs(func, **kwargs), data_split)
    #data = pd.concat(pool.map(WithExtraArgs(func, **kwargs), data_split))
    return data

def parallelize(data, func,pool,partition):
    data_split = [data[i:i + partition] for i in xrange(0, len(data), partition)]
    #data_split = np.array_split(data, partitions)
    data =pool.map(func, data_split)
    return data


In [3]:
cores = cpu_count() #Number of CPU cores on your system
partitions = cores
partitions

40

In [4]:
#!python -c 'import tensorflow as tf; print(tf.__version__)'
#!python -c 'import keras as kr; print(kr.__version__)'

# geohash feature vectors

In [5]:
geohash_map = pd.read_csv("../Traffic/Accidents/vectors/geohash_to_poi_vec.csv")

In [6]:
geohash_map.head()

,Geohash,Amenity,Bump,Crossing,Entrance,Give_Way,Junction,Noexit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Circle,Turning_Loop
0,9v6mn,47,0,2,0,0,2,1,0,0,11,0,0,10,157,0
1,9v6mm,45,0,0,0,0,0,1,0,0,0,0,0,42,314,0
2,9vg7s,3,0,0,0,0,0,3,0,0,0,0,0,1,12,0
3,9vg7t,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0
4,9vg7u,0,0,1,0,0,0,0,1,0,0,0,0,0,11,0


In [7]:
geohash_vec = geohash_map[[ u'Amenity', u'Bump', u'Crossing', u'Give_Way',
       u'Junction', u'Noexit', u'Railway', u'Roundabout', u'Station', u'Stop',
       u'Traffic_Calming', u'Traffic_Signal', u'Turning_Circle',
       u'Turning_Loop']]

In [8]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(geohash_vec.loc[:,'Amenity':]) 
scaled_values = scaler.transform(geohash_vec.loc[:,'Amenity':]) 
geohash_vec.loc[:,'Amenity':] = scaled_values
geohash_vec.head()

/apps/python/3.6-conda5.2/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Amenity,Bump,Crossing,Give_Way,Junction,Noexit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Circle,Turning_Loop
0,0.062500,0.0,0.000663,0.0,0.032787,0.005495,0.000000,0.0,0.024336,0.0,0.0,0.009533,0.341304,0.0
1,0.059840,0.0,0.000000,0.0,0.000000,0.005495,0.000000,0.0,0.000000,0.0,0.0,0.040038,0.682609,0.0
2,0.003989,0.0,0.000000,0.0,0.000000,0.016484,0.000000,0.0,0.000000,0.0,0.0,0.000953,0.026087,0.0
3,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.019565,0.0
4,0.000000,0.0,0.000332,0.0,0.000000,0.000000,0.002203,0.0,0.000000,0.0,0.0,0.000000,0.023913,0.0


In [9]:
geohash_dict={}
for index, row in geohash_map.iterrows():
    geohash_dict[row.Geohash] = np.array(geohash_vec.iloc[index])

In [10]:
import pickle
f = open("geo_vect_dict.pkl","wb")
pickle.dump(geohash_dict,f)
f.close()

In [11]:
import pickle
geo_dict = dict(zip(geohash_map.Geohash.unique(), range(len(geohash_map.Geohash.unique()))))
f = open("geo_dict.pkl","wb")
pickle.dump(geo_dict,f)
f.close()